In [1]:
# Import depedencies
from sportsreference.nfl.teams import Teams
from sportsreference.nfl.boxscore import Boxscores
import pandas as pd

In [35]:
# call boxscores (game stats) from sports-reference and create df

start_year = 2012
end_year = 2022
weeks = list(range(1, 53))
years = list(range(start_year, end_year + 1))

records = []
for y in years:
    for w in weeks:
        try:
            bs = Boxscores(week=w, year=y)
            data = bs.games[f"{w}-{y}"]
            for d in data:
                d['year'] = y
                d['week'] = w 
                records.append(d)
        except:
            pass

bs_df = pd.DataFrame(records)

In [37]:
bs_df.head()

,boxscore,away_name,away_abbr,away_score,home_name,home_abbr,home_score,winning_name,winning_abbr,losing_name,losing_abbr,year,week
0,201209050nyg,Dallas Cowboys,dal,24.0,New York Giants,nyg,17.0,Dallas Cowboys,dal,New York Giants,nyg,2012,1
1,201209090chi,Indianapolis Colts,clt,21.0,Chicago Bears,chi,41.0,Chicago Bears,chi,Indianapolis Colts,clt,2012,1
2,201209090cle,Philadelphia Eagles,phi,17.0,Cleveland Browns,cle,16.0,Philadelphia Eagles,phi,Cleveland Browns,cle,2012,1
3,201209090det,St. Louis Rams,ram,23.0,Detroit Lions,det,27.0,Detroit Lions,det,St. Louis Rams,ram,2012,1
4,201209090htx,Miami Dolphins,mia,10.0,Houston Texans,htx,30.0,Houston Texans,htx,Miami Dolphins,mia,2012,1


In [39]:
# call team stats from sports-reference and create df
teams = Teams()
team_dataframes = []
for t in teams:
    team_dataframes.append(t.dataframe)
    
teams_df = pd.concat(team_dataframes)

In [40]:
#can review individual team in team stats df
teams_df.loc['RAV']

abbreviation                                   RAV
defensive_simple_rating_system                -8.3
first_downs                                     55
first_downs_from_penalties                       5
fumbles                                          1
games_played                                     3
interceptions                                    2
losses                                           1
margin_of_victory                              7.3
name                              Baltimore Ravens
offensive_simple_rating_system                15.9
pass_attempts                                   88
pass_completions                                56
pass_first_downs                                32
pass_net_yards_per_attempt                     7.8
pass_touchdowns                                 10
pass_yards                                     735
penalties                                        8
percent_drives_with_points                    42.9
percent_drives_with_turnovers  

In [41]:
# Add team stats data to each line item of historic game data
records = []
for i, r in bs_df.iterrows():
    d = r.to_dict()
    
# team 1 is always home, team 2 is always away
    team_1_abv = r['home_abbr']
    team_2_abv = r['away_abbr']

    team_1_stats = teams_df.loc[team_1_abv.upper()]
    team_2_stats = teams_df.loc[team_2_abv.upper()]

    for k,v in team_1_stats.to_dict().items():
        d[f'team_1_{k}'] = v

    for k,v in team_2_stats.to_dict().items():
        d[f'team_2_{k}'] = v

    records.append(d)
    
combined_df = pd.DataFrame(records)

In [43]:
combined_df.head()

,boxscore,away_name,away_abbr,away_score,home_name,home_abbr,home_score,winning_name,winning_abbr,losing_name,...,team_2_rush_yards,team_2_rush_yards_per_attempt,team_2_simple_rating_system,team_2_strength_of_schedule,team_2_turnovers,team_2_win_percentage,team_2_wins,team_2_yards,team_2_yards_from_penalties,team_2_yards_per_play
0,201209050nyg,Dallas Cowboys,dal,24.0,New York Giants,nyg,17.0,Dallas Cowboys,dal,New York Giants,...,354,4.7,-4.2,-2.2,2,0.667,2,972,188,5.3
1,201209090chi,Indianapolis Colts,clt,21.0,Chicago Bears,chi,41.0,Chicago Bears,chi,Indianapolis Colts,...,313,4.0,-4.8,2.2,6,0.500,1,994,153,4.8
2,201209090cle,Philadelphia Eagles,phi,17.0,Cleveland Browns,cle,16.0,Philadelphia Eagles,phi,Cleveland Browns,...,451,4.4,38.9,26.9,1,1.000,3,1341,173,6.4
3,201209090det,St. Louis Rams,ram,23.0,Detroit Lions,det,27.0,Detroit Lions,det,St. Louis Rams,...,217,3.4,-9.0,-6.0,7,0.667,2,919,126,5.3
4,201209090htx,Miami Dolphins,mia,10.0,Houston Texans,htx,30.0,Houston Texans,htx,Miami Dolphins,...,192,3.3,12.4,6.1,2,1.000,3,1066,91,6.4


In [ ]:
combined_df.to_csv("match_team_data.csv", index=False)